### Import libraries and API Keys

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown
load_dotenv()


True

In [2]:

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")

# check if API keys are set
if not OPENAI_API_KEY:
    raise ValueError("Missing OpenAI API key")
if not GEMINI_API_KEY:
    raise ValueError("Missing Gemini API key")
if not PERPLEXITY_API_KEY:
    raise ValueError("Missing Perplexity API key")
if OPENAI_API_KEY:
    print("OpenAI API key is set")
if GEMINI_API_KEY:
    print("Gemini API key is set")
if PERPLEXITY_API_KEY:
    print("Perplexity API key is set")

OpenAI API key is set
Gemini API key is set
Perplexity API key is set


### Intiate Open AI Client

In [3]:
# client
openai_client = OpenAI(api_key=PERPLEXITY_API_KEY, base_url="https://api.perplexity.ai")

### Create message list for Open AI Client

In [4]:
# message list
message = "I want to generate support ticket for the customer who is facing issue with login to the application. "
message+= "The customer is using iOS app on iPhone 17 running iOS 26.0. "
message+= "The error message displayed is 'Invalid credentials'. "
message+= "Include device details and reate ticekt title"
message+= "Do not include any personal information about the customer. "
message+= "Do not include detailed description of the issue. And do not include any trobleshooting steps."
message+= "Generate the ticket with title and description only."

In [5]:
messages = [{"role": "user", "content": message}]

### Generate Ticket

In [6]:
#Generate ticket

openai_response = openai_client.chat.completions.create(
    model="sonar",
    messages=messages
)

ticket = openai_response.choices[0].message.content
display(Markdown(f"### Generated Ticket:\n{ticket}"))

### Generated Ticket:
### Ticket Title
**iOS App Login Issue: Invalid Credentials on iPhone 17 (iOS 26.0)**

### Description
Customer reports 'Invalid credentials' error during login attempt. Device: iPhone 17 running iOS 26.0 via iOS app.[2][3]

### Set Priority of the ticket

In [7]:
message = "Classify the priority of the following customer support ticket "
message += "The ticket is as follows: "+ ticket + " "
message += "Please classify the priority as either 'Low', 'Medium', or 'High'. "
message += "Respond with ticket title, ticket description and ticket priority level only."

messages = [{"role": "user", "content": message}]

In [10]:
openai_response = openai_client.chat.completions.create(
    model="sonar",
    messages=messages
)

priority = openai_response.choices[0].message.content 
#display(Markdown(f"### Generated Ticket:\n{ticket}"))
display(Markdown(f"### Ticket Priority:\n{priority}"))

### Ticket Priority:
### Ticket Title
**iOS App Login Issue: Invalid Credentials on iPhone 17 (iOS 26.0)**

### Description
Customer reports 'Invalid credentials' error during login attempt. Device: iPhone 17 running iOS 26.0 via iOS app.[2][3]

### Ticket Priority Level
**Medium**[1][2][4]

### Create 1st Response

In [13]:
# message list

message = "Create response email for the following customer support ticket."
message += "The ticket is as follows: "+ ticket + " "
message += "The email should be professional and customer friendly."
message += "Include only Subject and Body in the email."
message += "Do not include any greetings or sign-offs."
message += "Body should be concise and to the point."

messages = [{"role": "user", "content": message}]

In [14]:
openai_response = openai_client.chat.completions.create(
    model="sonar",
    messages=messages
)

response = openai_response.choices[0].message.content 
display(Markdown(f"### Email Response:\n{response}"))

### Email Response:
**Subject:** Help with iOS App Login Issue on iPhone 17 (iOS 26.0)

Thank you for reporting the "Invalid credentials" error during login on your iPhone 17 running iOS 26.0.[2][3]

To resolve this, please try these steps in order:

- Manually type your username and password instead of using autofill.[1]
- Update the app to the latest version via the App Store.[2]
- Go to **Settings > General > VPN & Device Management**, turn off any VPN profiles, then attempt login; if needed, delete and reinstall them later.[2]
- Reset your password if it may have changed or if issues persist.[1][2]
- Restart your device by quickly pressing volume up, then volume down, then holding the side button.[2]

If these do not work, reply with more details (e.g., does login succeed in a web browser?). We'll assist further.[5]

### Pass to Evaluator & Optimizer

We will use Gemini as Evaluator and Optimizer

In [15]:
gemini_client = OpenAI(api_key=GEMINI_API_KEY, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [20]:
# message list

message = "Take the input generated response email."
message += "The email is as follows: " + response + " "
message += "Evaluate the email for professionalism and customer friendliness."
message += "Analyze the response and suggest what can be improved in the email."
message += "Print the list of improvements only"
message += "Do not print the original email or any other text."
messages = [{"role": "user", "content": message}]

In [21]:
gemini_response = gemini_client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=messages
)

improv_list = gemini_response.choices[0].message.content 
display(Markdown(f"### List of Improvements:\n{improv_list}"))

### List of Improvements:
Here is a list of improvements for the email:

*   **Remove internal reference numbers:** The `[1][2][6]` and similar notations within the email text are internal references and appear unprofessional and confusing to the customer. They should be removed.
*   **Add a professional closing and signature:** The email ends abruptly. Include a professional closing such as "Sincerely," "Best regards," or "Kind regards," followed by the support team's name or company name.
*   **Enhance empathy and warmth in the body:** While the opening is good, the instructions are quite direct. Adding phrases like "We appreciate your patience as you try these steps" or "We're committed to helping you get this resolved" can make the tone more customer-friendly.
*   **Provide brief explanations for less common troubleshooting steps:** For steps like "Set date and time to automatic" or "Turn off or remove VPN," briefly explaining *why* these steps might help (e.g., "to ensure proper synchronization with our servers" or "VPNs can sometimes interfere with secure connections") can make the instructions clearer and more reassuring.
*   **Offer more detailed next steps for persistent issues:** While "reply to this email" is clear, you could add what to expect next (e.g., "If issues persist, please reply to this email, and we'll be happy to investigate further, perhaps requesting screenshots or more details about your setup.").

### Pass back to response client to improve the response

In [22]:
# message list

message = "Update the response email incorporating the improvements."
message += "Improvements suggested by optimizer-evaluator: " + improv_list + " "
message += "The ticket is as follows: "+ ticket + " "
message += "The email should be professional and customer friendly."
message += "Include only Subject and Body in the email."
message += "Do not include any greetings or sign-offs."
message += "Body should be concise and to the point."

messages = [{"role": "user", "content": message}]

In [23]:
openai_response = openai_client.chat.completions.create(
    model="sonar",
    messages=messages
)

response = openai_response.choices[0].message.content 
display(Markdown(f"### Email Response:\n{response}"))

### Email Response:
**Subject:** Steps to Resolve Your iOS App Login Issue on iPhone 17 (iOS 26.0)

Thank you for reaching out about the "Invalid credentials" error you're encountering when logging into our iOS app on your iPhone 17 running iOS 26.0. We're committed to helping you get this resolved quickly and appreciate your patience as you try these troubleshooting steps.

Here are some common fixes that often resolve this issue:

- **Update the app:** Ensure you're running the latest version from the App Store, as updates frequently address login bugs.
- **Restart your device:** A simple reboot can clear temporary glitches affecting authentication.
- **Set date and time to automatic:** Go to Settings > General > Date & Time and enable "Set Automatically." This ensures proper synchronization with our servers for secure login.
- **Turn off or remove VPN:** VPNs can sometimes interfere with secure connections. Disable any VPN apps temporarily and try logging in again.
- **Clear app cache or reinstall:** In the app settings, clear cache if available, or delete and reinstall the app to refresh your login data.

If these steps don't resolve the issue, please reply to this email with more details about your setup—such as screenshots of the error or any VPN/network info—and we'll investigate further right away.

Best regards,  
Support Team  
[Your Company Name]